In [2]:
# define an ordered dictionary data structure
# import the usual mid-level plotting objects
# import some basic low-level objects
# import periodic_table, which contains a DataFrame named elements

from collections import OrderedDict
from bokeh.plotting import output_file, show, figure
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.sampledata import periodic_table

# HoverTool implements the ability to provide information when the cursor
# "hovers" over a glyph. Most of its attributes specify user interactions,
# but the most important one is tooltips, which stores the information to
# display during hover. 

# ColumnDataSource is a data structure that you can attach to multiple
# graph elements 

In [32]:
periodic_table.elements.columns.tolist()

['atomic number',
 'symbol',
 'name',
 'atomic mass',
 'CPK',
 'electronic configuration',
 'electronegativity',
 'atomic radius',
 'ion radius',
 'van der Waals radius',
 'IE-1',
 'EA',
 'standard state',
 'bonding type',
 'melting point',
 'boiling point',
 'density',
 'metal',
 'year discovered',
 'group',
 'period']

In [267]:
for key in list(periodic_table.elements.keys()):
    print key


atomic number
symbol
name
atomic mass
CPK
electronic configuration
electronegativity
atomic radius
ion radius
van der Waals radius
IE-1
EA
standard state
bonding type
melting point
boiling point
density
metal
year discovered
group
period


In [268]:
# convert column 'atomic mass' to string

periodic_table.elements[
    'atomic mass'] = periodic_table.elements['atomic mass'].astype(str)

In [269]:
# Typically f-block elements are not assigned to any group, so they have "-"
# recorded in the 'group' column. The logical expression creates a filter
# that eliminates these elements

elements = periodic_table.elements[periodic_table.elements['group'] != "-"]

In [270]:
# define categorical scale for each axis

# group_range is a list of group numbers, e.g. ["1", "2", ..., "18"]
# period_range is a list of period numbers among those found in elements

# set(x) creates a set object, which contains the same items as x but
# without duplicates. sorted() turns it back into a list. reversed()
# because period # increases toward the bottom of the table.

group_range = [str(x) for x in range(1,19)]
period_range = [str(x) for x in reversed(sorted(set(elements['period'])))]

In [271]:
# associate a color with each element type (defined in column 'metal')

colormap = {
    "alkali metal"         : "#a6cee3",
    "alkaline earth metal" : "#1f78b4",
    "halogen"              : "#fdbf6f",
    "metal"                : "#b2df8a",
    "metalloid"            : "#33a02c",
    "noble gas"            : "#bbbb88",
    "nonmetal"             : "#baa2a6",
    "transition metal"     : "#e08e79",    
}

In [272]:
# For each element, define the following properties using elements:
# - group: group # as a string
# - period: period # as a string
# - symx: x pos of symbol  e.g., ['1:0.1','18:0.1','1:0.1',2':0.1', ...]
# - numbery: y pos of atomic number
# - massy: y pos of atomic mass
# - namey: y pos of element name
# - type_color: color hex value corresponding to element type

source = ColumnDataSource(
    data = dict(
        group = [str(x) for x in elements['group']],
        period = [str(y) for y in elements['period']],
        symx = [str(x) + ":0.1" for x in elements['group']],
        numbery = [str(x) + ":0.8" for x in elements['period']],
        massy = [str(x) + ":0.15" for x in elements['period']],
        namey = [str(x) + ":0.3" for x in elements['period']],
        sym = elements['symbol'],
        name = elements['name'],
        cpk = elements['CPK'],
        atomic_number = elements['atomic number'],
        electronic = elements['electronic configuration'],
        mass = elements['atomic mass'],
        type = elements['metal'],
        type_color = [colormap[x] for x in elements['metal']],
    )
)

# What's with the colons in the symx, numbery, massy, and namey?
# This is a little-known notation for specifying proportions of a
# category. For example, x = "USA:0.25" specifies 25% of the x-axis
# interval for the category "USA", measured from the left boundary
# of the category.

In [273]:
# specify file for output
# restrict tools available to the user, make hover available
# width and toolbar location could have been set in figure() as well

output_file("periodic.html")
TOOLS = "resize,hover,save"

p = figure(title = "Periodic Table", tools=TOOLS,
           x_range = group_range, y_range = period_range)

p.plot_width = 1200
p.toolbar_location = "left"

In [274]:
# add a Rectangle glyph with horizontal and vertical locations mapped
# to group and period columns of the source ColumnDataSource
# Width and height are 0.9 each. These properties are set implicitly
# by position, which isn't helpful to readers

p.rect("group", "period", 0.9, 0.9, source=source,
       fill_alpha = 0.6, color = "type_color")

# Alternative
# p.rect(x = "group", y = "period",
#       width = 0.9, height = 0.9,
#       source = source,
#       fill_alpha = 0.6,
#       color = "type_color")

# At this point, hovering produces a box displaying default information
# (index, graph coordinates, and pixel coordinates)

In [275]:
# provide a set of properties we'll use to format the hover box later

text_props = {
    "source" : source,
    "angle"  : 0,
    "color"  : "black",
    "text_align" : "left",
    "text_baseline" : "middle"
}

In [276]:
# add four text glyphs to display: chemical symbol, atomic number,
# name, and atomic mass
# The "**" notation signals that text_props is a dictionary, whose keys
# are parameter names and values are parameter values

p.text (x = "symx", y = "period",
        text = "sym",
        text_font_style = "bold",
        text_font_size = "15pt",
        **text_props)

p.text (x = "symx", y = "numbery", 
        text = "atomic_number",
        text_font_size = "9pt",
        **text_props)

p.text (x = "symx", y = "namey",
        text = "name",
        text_font_size = "6pt",
        **text_props)

p.text (x = "symx", y = "massy",
        text = "mass",
        text_font_size = "5pt",
        **text_props)

show(p)

In [277]:
# Remove the grid, which isn't useful for displaying a table

p.grid.grid_line_color = None

In [278]:
# hidden within the figure object is an anonymous data structure 
# describing the HoverTool. select() searches for and returns it
# dict(type=HoverTool) is equivalent to {"type" : HoverTool}

hover = p.select({"type":HoverTool})

In [279]:
# A "tooltip" is a box that pops up on a webpage to present infomration
# about whatever the cursor is hovering over. The tooltips attribute
# stores a data structure (list or dictionary) that specifies pairs of
# strings. The first of each pair is the label of a piece of information,
# and the second is a reference to where that information is stored.

# A "@" prefix refers to the figure's DataSource, while the "$" indicates
# a special property of area being hovered, such as the x or y coordinates
# or color. 

hover.tooltips = OrderedDict([
    ("name", "@name"),
    ("atomic number", "@atomic_number"),
    ("type", "@type"),
    ("atomic mass", "@mass"),
    ("CPK color", "$color[hex, swatch]:cpk"),
    ("electronic configuration", "@electronic"),
])

show(p)

In [1]:
periodic_table.elements

NameError: name 'periodic_table' is not defined